# Task
Refactor the notebook to replace the weather forecasting functionality with stock price retrieval. This involves installing `yfinance`, defining a `get_stock_price` tool using `yfinance` to calculate percentage gain over a specified duration, updating the tool declarations, and modifying the `run_groq` function to query and display stock price information for a given ticker and duration.

## Install yfinance

### Subtask:
Add the `yfinance` library to the pip install command to enable fetching stock data.


**Reasoning**:
The subtask is to add the `yfinance` library to the pip install command. I will modify the first code cell to include `yfinance` in the installation list.



In [31]:
!pip install -q mcp fastmcp httpx nest_asyncio groq yfinance

## Define get_stock_price tool

### Subtask:
Replace the `get_weather` function with a new `get_stock_price` function that uses `yfinance` to get historical stock data and calculate the gain over a specified duration. The function should accept `stock_ticker` (string) and `duration` (string, e.g., '1mo', '1y') as parameters.


**Reasoning**:
The subtask requires replacing the existing weather and alert tools with a new stock price tool. I will modify the relevant code cell to import `yfinance`, define the `get_stock_price` function, and remove the old functions.



In [104]:
import fastmcp
import httpx
import warnings
import yfinance as yf

warnings.filterwarnings("ignore")

mcp_server = fastmcp.FastMCP("stock_price")

@mcp_server.tool()
def get_stock_price(stock_ticker: str, duration: str = '1mo') -> str:
    """Calculates the percentage gain of a stock over a specified duration.

    Args:
        stock_ticker (str): The ticker symbol of the stock (e.g., 'AAPL', 'GOOG').
        duration (str): The duration for which to calculate the gain (e.g., '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max').

    Returns:
        str: A string indicating the stock's percentage gain or an error message.
    """
    try:
        ticker = yf.Ticker(stock_ticker)
        hist = ticker.history(period=duration)

        if hist.empty or len(hist) < 2:
            return f"Could not retrieve sufficient data for {stock_ticker} over {duration}."

        first_price = hist['Open'].iloc[0]
        last_price = hist['Close'].iloc[-1]

        if first_price == 0:
            return f"Cannot calculate percentage gain for {stock_ticker} due to zero initial price."

        percentage_gain = ((last_price - first_price) / first_price) * 100
        return f"{stock_ticker} gained {percentage_gain:.2f}% over {duration}."
    except Exception as e:
        return f"An error occurred while fetching stock price for {stock_ticker}: {e}"


**Reasoning**:
The previous step successfully defined the `get_stock_price` tool, despite a `DeprecationWarning` from an external library. The next logical step, as per the overall task, is to update the `function_declarations` to include the new `get_stock_price` tool and remove the old weather-related declarations.



In [105]:
import asyncio
from groq import Groq
import json
from fastmcp import Client as MCPClient
from fastmcp.client.transports import FastMCPTransport
from getpass import getpass

In [82]:
api_key = getpass("Enter your GROQ_API_KEY: ")

Enter your GROQ_API_KEY: ··········


In [106]:
function_declarations = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Calculates the percentage gain of a stock over a specified duration.",
            "parameters": {
                "type": "object",
                "properties": {
                    "stock_ticker": {
                        "type": "string",
                        "description": "The ticker symbol of the stock (e.g., 'AAPL', 'GOOG')."
                    },
                    "duration": {
                        "type": "string",
                        "description": "The duration for which to calculate the gain (e.g., '1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max').",
                        "default": "1mo"
                    }
                },
                "required": ["stock_ticker"]
            }
        }
    }
]

In [107]:
groq_client = Groq(api_key=api_key)
MODEL = "llama-3.3-70b-versatile" # Updated to a currently supported model
transport = FastMCPTransport(mcp_server)

In [115]:
async def chat(prompt):
    async with MCPClient(transport) as mcp_client:
        response = groq_client.chat.completions.create(
            model=MODEL,
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            tools=function_declarations,
            tool_choice="auto"
        )

        message = response.choices[0].message
        if message.tool_calls:
            call = message.tool_calls[0].function
            call_name = call.name
            call_args = json.loads(call.arguments)
        else:
            print("No function call; Groq said:", message.content)
            return

        # print("🔧 Groq wants:", call_name, call_args)

        result = await mcp_client.call_tool(call_name, call_args)
        print("###################################################")
        print("\n")
        print(result.content[0].text)
        print("\n")
        print("###################################################")


In [123]:
asyncio.get_event_loop().run_until_complete(chat("What is the return from AAPL stock in the last 3 months"))

###################################################


AAPL gained -0.05% over 3mo.


###################################################


In [124]:
asyncio.get_event_loop().run_until_complete(chat("What is the return from TATAPOWER.NS stock in the last 3 months"))

###################################################


TATAPOWER.NS gained -11.16% over 3mo.


###################################################
